In [48]:
import os

In [49]:
%pwd

'e:\\Final_year_project'

In [50]:
os.chdir("../")

In [51]:
%pwd

'e:\\'

In [65]:
from dataclasses import dataclass
from pathlib import Path
import yaml
from 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    
    local_data_file: Path
    unzip_dir: Path


SyntaxError: invalid syntax (940937813.py, line 4)

In [74]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import get_size, read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [54]:
import os
print(os.getcwd())  # This should point to your project root


e:\


In [55]:
import sys
sys.path.append(os.path.abspath("src"))


In [67]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


In [68]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size, read_yaml


In [69]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [71]:
import os
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import (DataIngestionConfig,
                                                PrepareBaseModelConfig,
                                                TrainingConfig,
                                                EvaluationConfig)


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    



    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/entbappy/chest-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

      --

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 94)

In [79]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-02-10 12:06:50,285: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-10 12:06:50,291: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-10 12:06:50,294: INFO: common: created directory at: artifacts]
[2025-02-10 12:06:50,297: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-10 12:06:50,303: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1hxjw3DiQuzBCIiehk5ZI0pGR62e6JJQT/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1hxjw3DiQuzBCIiehk5ZI0pGR62e6JJQT
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1hxjw3DiQuzBCIiehk5ZI0pGR62e6JJQT&confirm=t&uuid=e31de18c-5186-4136-ad6e-1414ccae38d5
To: E:\Final_year_project\End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:10<00:00, 4.87MB/s]

[2025-02-10 12:07:05,180: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1hxjw3DiQuzBCIiehk5ZI0pGR62e6JJQT/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


In [61]:
import os
os.chdir("E:/Final_year_project/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD")


In [62]:
import os

CONFIG_FILE_PATH = os.path.join(os.getcwd(), "config", "config.yaml")
PARAMS_FILE_PATH = os.path.join(os.getcwd(), "config", "params.yaml")
